# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile
import _pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import time
import copy

# Data loading
Remember to change path if needed

In [2]:
# # Take the data from Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# path = "/content/drive/My Drive/Colab Notebooks/Kaggle/Kaggle_data/"
# path_results = "/content/drive/My Drive/Colab Notebooks/Kaggle/Kaggle_results/"

path = "D:/GitHub/Machine-Learning/Kaggle/"
# path = "C:/Users/Javi/Documents/GitHub/Machine-Learning/Kaggle/"
path_results = path+"results/"

zf = zipfile.ZipFile(path+'zipped_TrainData.zip', 'r')
df_train = _pickle.loads(zf.open('TrainData.pkl').read())
zf.close()

zf = zipfile.ZipFile(path+'zipped_TestDataUnlabeled.zip', 'r')
df_test = _pickle.loads(zf.open('TestDataUnlabeled.pkl').read())
zf.close()

# Data split & Spectrum in regular bins
Data is split between a proper training set (later used in cross-validation), and a test_train set, which will help us in determining under/overfitting as we do have labels for them.

Spectrums are divided in regular size bins, always the same, so that we can treat them as features, not worrying about different mz scales. According to the literature the peaks contain the relevant information, then we only save the maximum value in the bin (range of mz coordinates) so that peak information is never lost. Moreover, by performing this regularization in bins, peaks at very close mz values (same compound, small mz differences due to experimental uncertainty) are seen by the machine as belonging to the same bin and therefore the same feature. Therefore, it facilitates to use peaks as values.

Also, peak values are normalized by the maximum peak value of the spectrum, as specific values are experiment-dependent and do not carry information, only the relation between peak sizes does.

In [3]:
def spectrum_in_bins(df,m,M,bin_size):
    # Now, let's define the mz ranges, and the label associated to each of them (the mean of the limiting values of each bin)
    range_min = []; range_max = []; range_label = [];
    for mz in range(m,M,bin_size):
        range_min.append(mz)
        range_max.append(mz+bin_size)
        range_label.append(np.mean([range_min[-1],range_max[-1]]).astype(int))


    N = len(df)  # number of samples
    L = len(range_min)  # length of new spectrum (number of bins)
    all_data = np.zeros((N,L))
    for idx in range(N): 
        intensity = df[['intensity']].iloc[idx].values[0]
        mzcoord   = df[['coord_mz']].iloc[idx].values[0]
        idx_data_in_bins = np.zeros((1,L))
        for i,mz in enumerate(range_min):
            intensity_range = intensity[(mzcoord > mz) & (mzcoord < (mz+bin_size))]
            if len(intensity_range) > 0 :
                idx_data_in_bins[0,i] = np.max(intensity_range)
            else: # if those mz coordinates are not in that spectrum
                idx_data_in_bins[0,i] = 0   

        # Normalize the amplitude of the spectrum
        idx_data_in_bins[0,:] = idx_data_in_bins[0,:] / np.max(idx_data_in_bins[0,:])
        
        
        all_data[idx,:] = idx_data_in_bins
    new_df = pd.DataFrame(data=all_data, columns = range_label, index = df.index)
    return new_df

In [4]:
def clean_nan_samples(spectrum,targets, c, cat):
# if there are any NaN values, we should remove those samples
    if (targets[cat].isnull().sum() > 0).all(): 
        merged = pd.concat([spectrum , targets],axis=1,copy=True)
        clean = merged.dropna(subset=[cat])
        print('Dropped ',len(merged)-len(clean))
        Y = clean.iloc[:,-9+c].to_numpy().reshape(-1,)
        X = clean.iloc[:,:-9]

    else:
        Y = targets.iloc[:,c].to_numpy().reshape(-1,)
        X = spectrum.copy(deep=True)
    return X , Y

In [5]:
# Hyperparameters of the spectrum processing
m = 2000; M = 20000; 
bin_size = 50;

In [6]:
# Change the way the spectre is handled: Now is MEAN
# Could be changed to the highest value of both or the lowest, susceptible to changes

def get_unique_spectre(spectre, df):
  # Include the ID_sample column for the group_by
  spectre['ID_sample'] = df.ID_sample
  # MEAN OF THE SPECTRE
  spectre = spectre.groupby('ID_sample').mean().reset_index()
  return spectre

In [7]:
def generate_csv_from_clf(clf_list, path, path_results, file_name):
  # classifiers must be provided with parameters, and in a list [clf_antibiotic0, clf_antibiotic1, ...]
  # spectrum and targets full train (containing all training points) will be used for training the clfs in clf list
  # df_test must be provided as loaded from the file

  # read all data from files
  zf = zipfile.ZipFile(path+'zipped_TrainData.zip', 'r')
  df_full_train = _pickle.loads(zf.open('TrainData.pkl').read());   zf.close()

  zf = zipfile.ZipFile(path+'zipped_TestDataUnlabeled.zip', 'r')
  df_test = _pickle.loads(zf.open('TestDataUnlabeled.pkl').read());   zf.close()

  # Process test df to get UNIQUE samples and convert to spectrum

  # df_unique_test = df_test.drop_duplicates(subset='ID_sample')

  spectrum_test_forcsv = spectrum_in_bins(df_test,m,M,bin_size)
  spectrum_test_forcsv = get_unique_spectre(spectrum_test_forcsv, df_test)

  # Process train set to later train the clfs
  spectrum_full_train = spectrum_in_bins(df_full_train.iloc[:,-2:],m,M,bin_size)
  targets_full_train  = df_full_train.iloc[:,1:-2]  

  # read the submission example file
  df_submission = pd.read_csv(path+'SubmissionSample.csv') 
  categories = df_submission.columns[1:]
  df_submission['ID']= spectrum_test_forcsv['ID_sample'].values
  # To eliminate the ID_sample from the spectrum
  spectrum_test_forcsv = spectrum_test_forcsv.drop(columns=['ID_sample'])
  for c, cat in enumerate(categories): 
      # clean NaN values
      X_train, Y_train = clean_nan_samples(spectrum_full_train,targets_full_train, c, cat)

      # fit the classifier
      clf_base = clf_list[c].fit(X_train,Y_train)
      # Compute its test prestiction and save this output
      o_test = clf_base.predict_proba(spectrum_test_forcsv)[:,1]
      df_submission[cat] = o_test

  # Save the dataframe with the predicted outputs
  df_submission = df_submission.set_index('ID')
  df_submission.to_csv(path_results + file_name + '.csv')
  print('DONE!')
  return df_submission

In [8]:
# Just to make a trial
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Let's create manually a list of good classifiers
clf_oxalacina       = SVC(kernel='rbf',class_weight='balanced', C=10000, gamma=0.1, probability=True)
clf_amikacina       = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100000)
clf_amoxiclav       = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100)
clf_ciprofloxacino  = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100)
clf_clindamicina    = KNeighborsClassifier(n_neighbors=16)
clf_eritromicina    = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=1000)
clf_levofloxacino   = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=10000)
clf_penicilina      = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=100000)
clf_tobramicina     = LogisticRegression(max_iter=1e6,solver='lbfgs',class_weight='balanced', C=10000)

clf_list = [clf_oxalacina, clf_amikacina, clf_amoxiclav, clf_ciprofloxacino, clf_clindamicina,
            clf_eritromicina, clf_levofloxacino, clf_penicilina, clf_tobramicina]

In [9]:
df_submission = generate_csv_from_clf(clf_list,path,path_results, 'Submission_24_12_19_mean_spectrum')

Dropped  88
Dropped  4
Dropped  82
DONE!


In [10]:
df_submission

,OXACILINA,AMIKACINA,AMOXI/CLAV,CIPROFLOXACINO,CLINDAMICINA,ERITROMICINA,LEVOFLOXACINO,PENICILINA,TOBRAMICINA
ID,,,,,,,,,
1081,0.612776,0.996739,0.877438,0.898146,0.1875,0.894030,0.922472,9.999988e-01,0.990395
1154,0.259207,0.049574,0.376553,0.304935,0.1875,0.253805,0.054409,9.955751e-01,0.215462
1225,0.280309,0.005619,0.328012,0.288620,0.2500,0.408429,0.174744,9.999803e-01,0.037936
124,0.359459,0.017563,0.477589,0.544543,0.2500,0.904559,0.295553,1.993916e-03,0.168926
1246,0.112013,0.007062,0.251122,0.365844,0.0000,0.421068,0.005400,9.990985e-01,0.045050
...,...,...,...,...,...,...,...,...,...
903,0.232318,0.007145,0.409132,0.508067,0.1875,0.496443,0.155544,9.999909e-01,0.045153
973,0.746764,0.999997,0.556959,0.763790,0.1250,0.458465,0.988598,1.501920e-14,0.999376
979,0.432437,0.000062,0.675015,0.716639,0.0625,0.761987,0.112098,9.999648e-01,0.026874
